In [1]:
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1704932445830_0004,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
#importando bibliotecas
import os
import sys
import pytz
import numpy as np
import datetime
import time
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import col, udf, lpad, translate, to_timestamp, date_format
from datetime import datetime
from datetime import timedelta
from datetime import date
from dateutil.relativedelta import relativedelta
from pyspark.sql.types import *
from pyspark.sql.functions import count, avg, col, asc, desc, min, avg,max,sum,count,stddev

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
#funcao freq
### FREQ FUNCTION
def Freq(pTabela,pColuna):

    qtd_total=pTabela.count()

    pTabela.registerTempTable("tab_input")
    frq = spark.sql(
            """
                select
                    {col},
                    count(*) as qtd_absoluto,
                    round(100*(count(*) / {tot}),2) as qtd_percentual
                from
                    tab_input
                group by
                    {col}
                order by
                    {col}
            """.format(col=pColuna, tot=qtd_total))

    qtd=frq.count()
    print('Quantidade de dominios',qtd)
    if qtd > 500:
        frq.show(1000,truncate=False)
        return "Dominio muito granular"

    else:
        frq.show(qtd,truncate=False)
        print("volumetria total:",qtd_total)
        return 'Freq da coluna ' + pColuna;

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
agora = datetime.now(pytz.timezone('America/Sao_Paulo'))
dthproc = agora.strftime("%Y%m%d%H%M%S")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
spark = SparkSession.builder.appName("t_ssp_montly_file").getOrCreate()
sqlContext = SQLContext(spark.sparkContext)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Carregando arquivos

In [6]:
### MOD_PER_FILE
file = "OcorrenciaMensal(Criminal)-EstadoSP_2023.csv"
#file = "OcorrenciaMensal(Criminal)-EstadoSP_2022.csv"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
ts_file_generation = file.split("-")[-1].replace(".csv", "")
print(ts_file_generation)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

EstadoSP_2023

In [8]:
# Endereco Buckets (provisorio)
bucket_control =    "bkt-ssp/Files/05_Control"
bucket_ingestion =  "bkt-ssp/Files/01_IngestionZone"
bucket_raw =        "bkt-ssp/Files/02_RawZone"
bucket_trusted =    "bkt-ssp/Files/03_TrustedZone"
bucket_refined =    "bkt-ssp/Files/04_RefinedZone"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
### MOD_PER_FILE
output_lake = "ingestion_ssp_2022"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
### MOD_PER_FILE
full_path_ingestion = 's3://{bkt}/{file}'.format(bkt=bucket_ingestion,file=file)
print(full_path_ingestion)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://bkt-ssp/Files/01_IngestionZone/OcorrenciaMensal(Criminal)-EstadoSP_2023.csv

In [11]:
start_time = time.time()

pouso = spark.read.format(
       "com.databricks.spark.csv").option(
       "header", "true").option(
       "encoding", "ISO-8859-1").option(
       "encoding", "UTF-8").option(
       "inferSchema", "false").option(
        "delimiter", ',').load(
       full_path_ingestion)

pouso.registerTempTable("pouso")
pouso.cache()
qtd = pouso.count()
print('records ingested: ', qtd)

# Calculate the elapsed time
elapsed_time = time.time() - start_time
print('Elapsed time:', elapsed_time, 'seconds')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

records ingested:  23
Elapsed time: 8.818820714950562 seconds

In [12]:
# Ordernamento dados ascendente
# Nao necessario para ingestao do ano 2023 e 2024
# pouso = pouso.orderBy(asc("ano"),asc("mes"), asc("regiao_ssp"), asc("id_municipio"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
# Listagem Colunas
for column in pouso.columns:
    print(column+',')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Natureza,
Janeiro,
Fevereiro,
Marco,
Abril,
Maio,
Junho,
Julho,
Agosto,
Setembro,
Outubro,
Novembro,
Dezembro,
Total,

In [14]:
### Lake SQL statement, partition files will be saved in raw bucket

lake = spark.sql(
    """
        select
            -- padrao para todas as tabelas

            --Nova Coluna para facilitar particionamento yyyy-mm
            
            --concat(ano, lpad(mes, 2, '0')) as yyyymm,
            --concat(ano, lpad(mes, 2, '0')) as yyyymm_partition,

            -- Data e Hora do Processamento

            {pdthproc} as ts_proc,
            {pdthproc} as ts_proc_partition,

            -- campos do arquivos
            
            Natureza,
            Janeiro,
            Fevereiro,
            Marco,
            Abril,
            Maio,
            Junho,
            Julho,
            Agosto,
            Setembro,
            Outubro,
            Novembro,
            Dezembro,
            Total
        from
            pouso
    """.format(tsfileger=ts_file_generation, pdthproc=dthproc))
lake.registerTempTable("lake")
lake.cache()
lake.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

23

In [15]:
#Destino
path_raw = os.path.join('s3://',bucket_raw,output_lake)
print(path_raw)

lake.coalesce(1).write \
.partitionBy("natureza", "ts_proc", "ts_proc_partition") \
.csv(path_raw,mode='overwrite')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://bkt-ssp/Files/02_RawZone/ingestion_ssp_2022

In [18]:
#End

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…